# Easy Functional Data Engineering Examples - A DAG of partitions

In [13]:
# Let's get our dependencies
import sys, os
!{sys.executable} -m pip install pandas
import pandas as pd
import sqlite3
import shutil
from datetime import datetime

# some constants
DATA_FILE_PATH_1="data_01_01.csv"
DATA_FILE_PATH_2="data_01_02.csv"
STAGING_AREA="staging.csv"

In [3]:
df = pd.read_csv(DATA_FILE_PATH_1)
print("This is our dataframe...from day 01_01: \n", df.head(5))

This is our dataframe...from day 01_01: 
    Value type
0      1    a
1      2    b
2      3    c


**Day 1**: We're dumping this data into our "staging area" which we have in a file called "staging.csv" and then we process it further. Think of this as moving stuff from an external source to our data domain.

In [6]:
df.to_csv(STAGING_AREA)

Next, we're taking our staging area and aggregate (by summing) by "a" and by "b". 

In [8]:
df=pd.read_csv(STAGING_AREA)
df

,Unnamed: 0,Value,type
0,0,1,a
1,1,2,b
2,2,3,c


In [9]:
df.groupby('type').sum()

,Unnamed: 0,Value
type,,
a,0,1
b,1,2
c,2,3


**Day 2**: Perfect, so now it's day 2; Let's add more data to our staging area and calculate our aggregations again...

In [11]:
df = pd.read_csv(DATA_FILE_PATH_2)
df.to_csv(STAGING_AREA, mode="a", header="False")

Now let's group again

In [12]:
df=pd.read_csv(STAGING_AREA)
df.groupby('type').sum()

,Unnamed: 0,Value
type,,
a,0,4
b,2,6
c,2,3


**We have a problem**: Oh no, the CEO just called, he just got the report from yesterday and he's confused. He's really worried about the numbers for "a".

Can you check whether these numbers are ok? 

**Problem**: No, not easily. Because we don't know the inputs for yesterday. After all, we just keep on dumping stuff into out staging area here and then run the same "compute" functionality. If we run the compute functionality, it returns what we have above. 

**Solution**: So let's see whether we can reorganize that into a "DAG of tasks" one where we know how to compute just one partition, and one partition with specific inputs!

In [18]:
### WIP: Need to load immutable based on input ;)) ### Not happening yet...
def load_to_staging_area(filepath: str):
    df = pd.read_csv(filepath)
    df['source'] = filepath
    from os.path import exists
    df.to_csv(STAGING_AREA, mode="a", header="False") if os.path.exists(STAGING_AREA)  \
        else df.to_csv(STAGING_AREA) 

load_to_staging_area(DATA_FILE_PATH_1)

True